In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
%aimport datagen

In [ ]:
import numpy as np
import scipy
import xarray as xr
import pandas as pd


In [ ]:
import logging
logger = logging.getLogger(__name__)

In [ ]:
d0 = xr.DataArray(
                  data=[1,2,3],
            dims='axis',
            coords=[['x','y','z']], 
        name='sigma'
    
)

In [ ]:
d0.dims

In [ ]:
d0s = d0.to_series()
d0s

In [ ]:
print(d0)

In [ ]:
d0

In [ ]:
d0.sel(sigma='x')

In [ ]:
d0.to_series()

In [ ]:
for val in d0:
    print(val)

In [ ]:
d0.data*np.atleast_2d(d0.data).T

In [ ]:
coords=['x','y','z']
coords = [coords,coords,coords]
d1 = xr.DataArray(np.arange(3*3*3).reshape(3,3,3), 
#              dims=('axis0','axis1'), 
                  dims=('i','j','k'), 
#              coords={'i':coords,'j':'coords'}
             coords=coords,
            name='covariance'
            )

In [ ]:
d1

In [ ]:
len(d1.dims)

In [ ]:
c = d1.coords
c

In [ ]:
df['x']

In [ ]:
d1[0]

In [ ]:
df

In [ ]:
d1

In [ ]:
d1[0]

In [ ]:
d1_slice=d1[:,0]

d1_slice.coords.keys()

In [ ]:
df.iloc[0]

In [ ]:
df.iloc[:, 0]

In [ ]:
d1.values

In [ ]:
d1.to_dataframe().unstack()

In [ ]:
for val in d1:
    print(val)

In [ ]:
d.loc['z','y']

In [ ]:
d[2,1]

In [ ]:
d.loc[:,'z']

In [ ]:


def build_covariance_matrix(sigmas, correlations):
    """
    Builds a covariance matrix from sigmas and correlations.

    Args:
        sigmas (pandas.Series): Series mapping parameter name to std. dev.
        correlations (dict): Mapping of tuples to correlations, e.g.
            ``{(param1,param2):0.5}``. Default correlation for unspecified
            pairs is zero.
    Returns:
        (DataFrame): Covariance matrix.
    """
    cov = pd.DataFrame(index=sigmas.index,
                       columns=sigmas.index,
                       data=np.diag(sigmas ** 2),
                       dtype=np.float
                       )
    for param_pair, pair_corr in correlations.items():
        p1, p2 = param_pair
        pair_cov = pair_corr * sigmas[p1] * sigmas[p2]
        cov.loc[p1, p2] = cov.loc[p2, p1] = pair_cov

    try:
        scipy.linalg.cholesky(cov,
                              lower=True)
    except np.linalg.LinAlgError:
        logger.error("Provided correlation values result in a covariance"
                     " which is not positive semidefinite.")
        raise

    return cov
